In [ ]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

###   Importar Modelo entrenado

In [ ]:

modelo = joblib.load('modelo_entrenado.plk')


### Preprocesaniento de datos

Cuando se llama a la función preprocessing se realiza una limpieza y transformacion de los datos
luego de escalar los datos devuelve dos vectores con los datos para la prediccion

In [ ]:


time_step = 120
scaler = MinMaxScaler( feature_range=(0, 1) )

def preprocessing(data):
  data = data.drop(['Date','Adj Close'], axis=1)
  data = data.round(2)

  #transform dataframe to numpy array
  data = data.values

  #normalize data
  data = scaler.fit_transform(data)

  X_train = []
  y_train = []

  for i in range(time_step, len(data)):
      X_train.append(data[i-time_step:i, :])
      y_train.append(data[i, 0])
 
  X_train, y_train = np.array(X_train), np.array(y_train)
  


  return X_train, y_train



### Lectura de los datos y predicción

In [ ]:

df = pd.read_csv('AMZN.csv')

X_df, y_df = preprocessing(df)

real = df.loc[time_step:, ['Open']].values.round(2)

nnr = modelo.predict(X_df)


nnr_list = []

for i in range(len(nnr)):
    x = (nnr[i,0]*(scaler.data_max_[0]-scaler.data_min_[0]))+scaler.data_min_[0]
    nnr_list.append(x)



### Contrastación de los datos predichos respecto a los datos reales

In [ ]:


plt.plot(real, color = 'red', label = 'Real Stock Price')
plt.plot(nnr_list, color = 'blue', label = 'Predicted Stock Price')
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

se reestructura la lista para que tenga el mismo formato que el precio real

In [ ]:
nnr_list = np.array(nnr_list)


nnr_list = nnr_list.reshape(-1,1)

Error absoluto medio de la predicción.

In [ ]:
mean_absolute_error(nnr_list, real)

El porcentaje de error sobre el valor de las acciones.

In [ ]:
mean_absolute_error(nnr_list, real) / np.mean(real)

#### Calculamos el error de cada dia y lo graficamos.

In [ ]:
error = nnr_list - real

plt.plot(error)
plt.axhline(y=0, xmin=0, xmax=20, ls='--', color='k')
plt.title('Contraste entre prediccion y real')
plt.xlabel('Tiempo')
plt.ylabel('Error de predicción')